In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
b_s = 3
logit_num = 4
logits = torch.rand(b_s, logit_num)
mlogits = torch.rand(b_s, logit_num)

In [3]:
logits, mlogits

(tensor([[0.5767, 0.6412, 0.2021, 0.6762],
         [0.6728, 0.4351, 0.5528, 0.0493],
         [0.4224, 0.9186, 0.2699, 0.3113]]),
 tensor([[0.3207, 0.9544, 0.5005, 0.3938],
         [0.9448, 0.2312, 0.1109, 0.5175],
         [0.8904, 0.4951, 0.7455, 0.1199]]))

In [4]:
def cos_both_dim(logits_student, logits_teacher):
    cos_1 = 1 - torch.nn.functional.cosine_similarity(logits_student, logits_teacher, dim=-1)
    cos_2 = 1 - torch.nn.functional.cosine_similarity(logits_student, logits_teacher, dim=-2)
    
    return cos_1.mean() + cos_2.mean()

def cos_both_dim_prod(logits_student, logits_teacher):
    cos_1 = 2 - torch.nn.functional.cosine_similarity(logits_student, logits_teacher, dim=-1)
    cos_2 = 2 - torch.nn.functional.cosine_similarity(logits_student, logits_teacher, dim=-2)
    product = torch.outer(cos_1,cos_2)
    return product.mean()

def cos_ma_interactive_cd(logits_student, logits_teacher):
    #print(logits_student.shape)
    student_class_ineraction_a = logits_student.unsqueeze(0)
    student_class_ineraction_b = logits_student.unsqueeze(1)
    student_class_ineraction_prod = student_class_ineraction_a * student_class_ineraction_b
    
    teacher_class_ineraction_a = logits_teacher.unsqueeze(0)
    teacher_class_ineraction_b = logits_teacher.unsqueeze(1)
    teacher_class_ineraction_prod = teacher_class_ineraction_a * teacher_class_ineraction_b
    #cos_1 = 2 - torch.nn.functional.cosine_similarity(logits_student, logits_teacher, dim=-1)
    #cos_2 = 2 - torch.nn.functional.cosine_similarity(logits_student, logits_teacher, dim=-2)
    #product = torch.outer(cos_1,cos_2)
    #print(student_class_ineraction_prod,teacher_class_ineraction_prod)
    cos_class_interaction =1 - torch.nn.functional.cosine_similarity(student_class_ineraction_prod
                                                                  , teacher_class_ineraction_prod, dim=2)
    #print(cos_class_interaction)
    return cos_class_interaction.mean() #should i square these in place? i only did it cuz i saw zeros...idk
# i think i just do this additively to the previous 2.

In [7]:
l_1 = torch.tensor([[3,2,1],[4,5,6],[7,11,9]]).to(torch.float32)
l_2 = torch.tensor([[6,4,2],[8,10,12],[3.5,5.5,4.5]]).to(torch.float32)

In [8]:
cos_ma_interactive_cd(l_1,l_2)

tensor(-2.6491e-08)

In [9]:
cos_both_dim(l_1,l_2)

tensor(0.1900)

In [10]:
cos_ma_interactive_cd(logits,logits), cos_both_dim(logits,logits)

(tensor(-3.9736e-08), tensor(-9.4374e-08))

In [8]:
x = torch.tensor([[1.0,2,3],[2,3,4]])
y = torch.tensor([[1.1,2.7,2.6],[3,3.2,3.3]])

In [142]:
print(x.shape)

torch.Size([2, 3])


In [159]:
#all interactions:
def all_interactions_cd(logits_student, logits_teacher):
    student_batch_interactions = logits_student.unsqueeze(1) * logits_student.unsqueeze(0)
    student_class_interactions = logits_student.unsqueeze(2) * logits_student.unsqueeze(1)

    student_batch_interactions = student_batch_interactions.unsqueeze(-1)  # Shape becomes [b, b, c, 1]
    student_class_interactions = student_class_interactions.unsqueeze(0)  # Shape becomes [1, b, c, c]

    student_expanded = student_batch_interactions * student_class_interactions
    
    teacher_batch_interactions = logits_teacher.unsqueeze(1) * logits_teacher.unsqueeze(0)
    teacher_class_interactions = logits_teacher.unsqueeze(2) * logits_teacher.unsqueeze(1)

    teacher_batch_interactions = teacher_batch_interactions.unsqueeze(-1)  # Shape becomes [b, b, c, 1]
    teacher_class_interactions = teacher_class_interactions.unsqueeze(0)  # Shape becomes [1, b, c, c]

    teacher_expanded = teacher_batch_interactions * teacher_class_interactions
    
    cd_0 =1 - torch.nn.functional.cosine_similarity(student_expanded, teacher_expanded, dim=0)
    cd_1 =1 - torch.nn.functional.cosine_similarity(student_expanded, teacher_expanded, dim=1)
    cd_2 =1 - torch.nn.functional.cosine_similarity(student_expanded, teacher_expanded, dim=2)
    cd_3 =1 - torch.nn.functional.cosine_similarity(student_expanded, teacher_expanded, dim=3)
    #print(cd_0.mean(), cd_1.mean(), cd_2.mean(), cd_3.mean())
    return cd_0.mean() + cd_1.mean() + cd_2.mean() + cd_3.mean()

In [164]:
ttt = all_interactions_cd(logits,mlogits)

In [165]:
ttt


tensor(2.0156)

In [110]:
ttt[0][1][0][:]

tensor([2., 4., 6.])

In [90]:
def axes_interactions_loss(logits_student, logits_teacher, axis = 1, cd_collapse_axis = 0):
    #do 1 for batches, 2 for classes.
    student_expanded_a = logits_student.unsqueeze(axis)
    student_expanded_b = logits_student.unsqueeze(axis - 1)
    student_expanded = student_expanded_a * student_expanded_b
    
    teacher_expanded_a = logits_teacher.unsqueeze(axis)
    teacher_expanded_b = logits_teacher.unsqueeze(axis - 1)
    teacher_expanded = teacher_expanded_a * teacher_expanded_b
    
    cd = 1 - torch.nn.functional.cosine_similarity(student_expanded, teacher_expanded, dim=cd_collapse_axis)

    return cd.mean()

In [92]:

x,axes_interactions_loss(x,y,2,1)

torch.Size([2, 3, 3])
torch.Size([2, 3])


(tensor([[1., 2., 3.],
         [2., 3., 4.]]),
 tensor(0.0237))

In [50]:
x[1]

tensor([2., 3., 4.])